# Дел 1: пребарување на Макстат

In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

MAKSTAT_URL = "https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/search"

Основна функција за пребарување:

In [53]:
def keyword_search(keyword):
    params = {
        "searchquery": keyword
    }
    r = requests.get(MAKSTAT_URL, params=params)
    r.raise_for_status()
    return r.text   # HTML

if __name__ == "__main__":
    html = keyword_search("население")
    soup = BeautifulSoup(html, "html.parser")
    print(soup.prettify())

<!DOCTYPE html>
<html lang="mk">
 <head>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   База на податоци МАКСТАТ
  </title>
  <link href="/PXWeb/Resources/Styles/reset.css?v=22.1.0.23905" media="screen" rel="stylesheet" type="text/css"/>
  <link href="/PXWeb/Resources/Styles/main-common.css?v=22.1.0.23905" media="screen" rel="stylesheet" type="text/css"/>
  <link href="/PXWeb/Resources/Styles/main-pxweb.css?v=22.1.0.23905" media="screen" rel="stylesheet" type="text/css"/>
  <link href="/PXWeb/Resources/Styles/jQuery/redmond/jquery-ui.css?v=22.1.0.23905" media="screen" rel="stylesheet" type="text/css"/>
  <link href="/PXWeb/Resources/Styles/jQuery/tablesorter/style.css?v=22.1.0.23905" media="screen" rel="stylesheet" type="text/css"/>
  <link href="/PXWeb/Resources/Styles/main-custom.css?v=22.1.0.23905" media="screen" rel="stylesheet" type="text/css"/>
  <script src="/PXWeb/Resources/Scripts/jquery-3.5.1.min.js?v=22.1.0.23905">
  </script>
  <script 

Проблем: горенаведеното пребарување честопати дава премногу опции.

Затоа, би помогнало ако ги ограничиме резултатите на оние кои го содржат зборот во насловот на табелата.

Како го правиме ова?
- прво праќаме GET request за да го добиеме првичниот HTML и да ги утврдиме скриените инпут полиња (пр. __VIEWSTATE, __EVENTVALIDATION) кои потоа ќе ни бидат потребни за вториот POST request
- потоа праќаме POST request со потребните филтри за да се пребаруваат само насловите според keyword
- ги зачувуваме насловот, линкот до табелата, и датумот кога податоците биле објавени.

In [54]:
def keyword_search(keyword):
    session = requests.Session()

    # Step 1: initial GET
    r = session.get(MAKSTAT_URL, params={"searchquery": keyword})
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    # Step 2: collect hidden inputs
    data = {}
    for inp in soup.select("input[type=hidden]"):
        if inp.get("name"):
            data[inp["name"]] = inp.get("value", "")

    # Step 3: add your desired options
    data["ctl00$ContentPlaceHolderMain$pxSearch$txtSearch"] = keyword
    data["ctl00$ContentPlaceHolderMain$pxSearch$cmdSearch"] = "Пребарај"
    data["ctl00$ContentPlaceHolderMain$searchOptions"] = "select"   # "Пребарај само"
    data["ctl00$ContentPlaceHolderMain$chkTitle"] = "on"            # "Наслов на табелата"

    # Step 4: POST back
    r2 = session.post(MAKSTAT_URL, data=data)
    r2.raise_for_status()

    # wrap in soup
    soup2 = BeautifulSoup(r2.text, "html.parser")

    rows = soup2.select("tr")
    results = {"titles": [], "links": [], "published": []}

    for row in rows:
        a = row.select_one("td.searchCellTable a")
        pub = row.select_one("td.searchCellPublished")
        if a and pub:
            results["titles"].append(a.get_text(strip=True))
            results["links"].append("https://makstat.stat.gov.mk" + a["href"])
            results["published"].append(pub.get_text(strip=True))

    return results

if __name__ == "__main__":
    data = keyword_search("трговија")
    df = pd.DataFrame.from_dict(data)
    print(df)
    for t, l, p in zip(data["titles"], data["links"], data["published"]):
        print(f"{p} – {t} → {l}")


                                              titles  \
0  Е-трговија кај деловните субјекти од нефинанси...   
1  Промет во трговијата со моторни возила, тргови...   

                                               links  published  
0  https://makstat.stat.gov.mk/PXWeb/pxweb/mk/Mak...  08.4.2025  
1  https://makstat.stat.gov.mk/PXWeb/pxweb/mk/Mak...  01.9.2025  
08.4.2025 – Е-трговија кај деловните субјекти од нефинансискиот сектор, со 10 или повеќе вработени (%) → https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__InfOpstestvo__DelovniSubjekti/325_InfOpst_Mk_12Entecom_ml.px/
01.9.2025 – Промет во трговијата со моторни возила, трговија на големо и на мало, по НКД Рев. 2, по месеци → https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__VnatresnaTrgovija__VTBazna2021/125_Vt_mk_11_TrgDej_2021_ml.px/


In [55]:
df = pd.DataFrame.from_dict(data)
df

,titles,links,published
0,Е-трговија кај деловните субјекти од нефинанси...,https://makstat.stat.gov.mk/PXWeb/pxweb/mk/Mak...,08.4.2025
1,"Промет во трговијата со моторни возила, тргови...",https://makstat.stat.gov.mk/PXWeb/pxweb/mk/Mak...,01.9.2025


In [56]:
data['links'][0]

'https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__InfOpstestvo__DelovniSubjekti/325_InfOpst_Mk_12Entecom_ml.px/'

Линк од резултатите на пребарувањето:
```
https://makstat.stat.gov.mk/PXWeb/pxweb/mk/MakStat/MakStat__InfOpstestvo__DelovniSubjekti/325_InfOpst_Mk_12Entecom_ml.px/
```

Линк од API endpoint:
```
https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/InfOpstestvo/DelovniSubjekti/325_InfOpst_Mk_12Entecom_ml.px
```



In [29]:
API_BASE_URL = "https://makstat.stat.gov.mk:443/PXWeb/api/v1/"

test_url = data['links'][0]
API_suffix = test_url.replace("MakStat__", "").replace("__", "/").split("pxweb/")[1]
print(API_suffix)

mk/MakStat/InfOpstestvo/DelovniSubjekti/325_InfOpst_Mk_12Entecom_ml.px/


In [57]:
import requests
import json

TABLE_URL = API_BASE_URL + API_suffix

print(TABLE_URL)

query = {
  "query": [],
  "response": {
    "format": "px"
  }
}

TEST_URL = "https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/InfOpstestvo/DelovniSubjekti/325_InfOpst_Mk_12Entecom_ml.px"

response = requests.post(TABLE_URL, json=query)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
else:
    print(f"Request failed with status code: {response.status_code}")

https://makstat.stat.gov.mk:443/PXWeb/api/v1/mk/MakStat/InfOpstestvo/DelovniSubjekti/325_InfOpst_Mk_12Entecom_ml.px/
Request successful!


In [58]:
!pip install pyaxis

In [60]:
from pyaxis import pyaxis

with open("result.px", 'wb') as f:
    f.write(response.content)

result = pyaxis.parse('result.px', encoding='utf-8')#, 'ISO-8859-5')

table_df = pd.DataFrame(result['DATA'])

table_df.to_markdown()

Multilingual PX file


'|    | Деловни субјекти (10+) кои имале е-трговија                                                               |   Година | DATA   |\n|---:|:----------------------------------------------------------------------------------------------------------|---------:|:-------|\n|  0 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2006 | 14.6   |\n|  1 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2007 | 7.9    |\n|  2 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2008 | 9.8    |\n|  3 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2009 | 7.1    |\n|  4 | Деловни субјекти кои нарачале преку интернет или други компјутерски посредувани мрежи (е-купување)        |     2010 | 6.8    |\n|  5 | Деловни субјекти кои нарачале преку инте

In [61]:
table_df

,Деловни субјекти (10+) кои имале е-трговија,Година,DATA
0,Деловни субјекти кои нарачале преку интернет и...,2006,14.6
1,Деловни субјекти кои нарачале преку интернет и...,2007,7.9
2,Деловни субјекти кои нарачале преку интернет и...,2008,9.8
3,Деловни субјекти кои нарачале преку интернет и...,2009,7.1
4,Деловни субјекти кои нарачале преку интернет и...,2010,6.8
5,Деловни субјекти кои нарачале преку интернет и...,2011,8.1
6,Деловни субјекти кои нарачале преку интернет и...,2012,5.1
7,Деловни субјекти кои нарачале преку интернет и...,2013,4.6
8,Деловни субјекти кои нарачале преку интернет и...,2014,5.0
9,Деловни субјекти кои нарачале преку интернет и...,2015,5.9


# Дел 2: евалуација

Евалуација звучи како дел кој се одвива после имплементацијата, но клучно е да се направи што е можно порано, поради две главни причини:
- утврдува мерлива цел, открива скриени потешкотии,
- **овозможува оптимизација на перформанс *за време на имплементацијата***.

In [62]:
system_prompt = """

Твојата задача е да генерираш прашања и клуч на точни одговори врз основа на табела која ќе ти биде дадена.

За секоја табела, треба да одговориш во JSON format со следниве полиња:

- question: прашањето, на македонски јазик, во врска со некој податок од табелата
- answer: точниот одговор на прашањето

За секоја табела, генерирај еден пар од прашање и одговор.
"""


import requests
import json
import os
from google.colab import userdata

url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
  "Authorization": f"Bearer {userdata.get('OPENROUTER_API_KEY')}",
  "Content-Type": "application/json"
}



def generate_synthetic_data(tables, N=10):
    """
    Params:
    - tables: list of table (in pandas DataFrame format) to generate synthetic data for
    - N: number of synthetic data points to generate for each table
    """

    QA_pairs = []
    for table in tables:
        payload = {
        "model": "google/gemini-2.5-flash",
        "messages": [
            {
            "role": "system",
            "content": system_prompt
            },
            {
            "role": "user",
            "content": table.to_markdown()
            }
        ]
        }

        response = requests.post(url, headers=headers, json=payload)
        pair = response.json()['choices'][0]['message']['content']
        QA_pairs.append(pair)

    return QA_pairs


eval_set = generate_synthetic_data([table_df])

eval_set

['```json\n{\n  "question": "Колку деловни субјекти кои примиле нарачки преку интернет или други компјутерски посредувани мрежи (е-продажба) имало во 2017 година?",\n  "answer": "6.0"\n}\n```']

In [64]:
import json

for i in eval_set:
    # Step 1: Strip the outer list and ```json markers
    cleaned = i.strip("['```json\n").rstrip("\n```']")

    # Step 2: Parse the JSON
    data = json.loads(cleaned)

    #eval_df = pd.DataFrame.from_dict(data)

data


{'question': 'Колку деловни субјекти кои примиле нарачки преку интернет или други компјутерски посредувани мрежи (е-продажба) имало во 2017 година?',
 'answer': '6.0'}

Следни чекори:

1. Интегрирање на алатките и креирање функционален агент. Цел: прототип каде што корисник пишува прашање -> агентот пребарува МАКСТАТ со клучен збор -> добива листа на табели -> избира табела -> табелата е преземена и претворена во DataFrame

2. Креирање на синтетички сет за евалуација (100 прашања), кој се состои од `question`, `search_keyword`, `table_name`, `sql_query`, `answer`.

Дизајн одлуки околу евалуациското множество:

- Системската порака за моделот кој ги генерира синтетичките податоци треба да разјасни дека `seach_keyword` мора да биде содржан во `table_name`. Ова овозможува да го увидиме перформансот на моделот кој пребарува ако функцијата за пребарување секогаш ги филтрира резултатите со "само наслов".

- за некои прашања, `answer` е integer или float ("колку деловни субјекти кои направиле X имало во 2017 година"). За други прашања, `answer` е под-табела (ако корисникот праша "дај ми ги податоците за X од Гостивар"). Во вториов случај, треба да се евалуира и/или оптимизира моделот врз основа на конкретното SQL query, а целиот процес врз основа на резултатот.

- Множеството треба да овозможи изолирано тестирање на различните делови од процесот. На пример, може една евалуација да биде кој модел е најдобар во извлекување на правилната `table_name` според даденото `question`; друга евалуација да биде колку добар е моделот во давање на `answer` врз основа на одредена `table_name`; а ултимативната евалуација, секако, е перформансот во давање на `answer` базиран на `question`.


Следен состанок: оптимизација со [DSPy](https://dspy.ai/api/optimizers/MIPROv2/).